In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from flask import Flask, request, jsonify


In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from flask import Flask, request, jsonify
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

In [ ]:
!pip install pandas scikit-learn
from sklearn.utils import resample

# **Loading Dataset**

In [ ]:
file_path = "/content/all-data.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1", header=None)
df.columns = ["Sentiment", "Text"]
df["Sentiment"] = df["Sentiment"].map({"neutral": 0, "positive": 1, "negative": -1})
df.head()

,Sentiment,Text
0,0,"According to Gran , the company has no plans t..."
1,0,Technopolis plans to develop in stages an area...
2,-1,The international electronic industry company ...
3,1,With the new production plant the company woul...
4,1,According to the company 's updated strategy f...


**Sentimnet Count**

In [ ]:
df["Sentiment"].value_counts()

,count
Sentiment,
0,2879
1,1363
-1,604


# **Balancing Dataset**

In [ ]:

df.columns = ["Sentiment", "Text"]

sentiment_mapping = {-1: "negative", 0: "neutral", 1: "positive"}
df["Sentiment"] = df["Sentiment"].map(sentiment_mapping
df_neutral = df[df["Sentiment"] == "neutral"]
df_positive = df[df["Sentiment"] == "positive"]
df_negative = df[df["Sentiment"] == "negative"]


In [ ]:
df_positive_upsampled = resample(df_positive, replace=True, n_samples=2879, random_state=42)
df_negative_upsampled = resample(df_negative, replace=True, n_samples=2879, random_state=42)


In [ ]:
df_balanced = pd.concat([df_neutral, df_positive_upsampled, df_negative_upsampled])

In [ ]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


**Reading Balnace Data**

In [ ]:

df_balanced.to_csv("balanced_data.csv", index=False)

# Check new class distribution
print(df_balanced["Sentiment"].value_counts())

Sentiment
neutral     2879
negative    2879
positive    2879
Name: count, dtype: int64


In [ ]:

df_balanced = pd.read_csv("balanced_data.csv")
# Print first few rows
print(df_balanced.head())

  Sentiment                                               Text
0   neutral  The deliveries are scheduled for the summer an...
1  negative  Ruukki Group calculates that it has lost EUR 4...
2  positive  Savon koulutuskuntayhtyma , Finland based comp...
3   neutral  According to the Act , universities can now ac...
4  positive  Finnish engineering and technology company Met...


**Text Pre Processing**

In [ ]:
nltk.download('stopwords')

# Load balanced dataset
df = pd.read_csv("balanced_data.csv")

# Text preprocessing function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply text preprocessing
df["Text"] = df["Text"].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Training And Testing and applying Logistic Regression Model**

In [ ]:
X = df["Text"]
y = df["Sentiment"]
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
X_tfidf = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")



Model Accuracy: 0.88


**Classification Report**

In [ ]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

    negative       0.90      0.97      0.93       568
     neutral       0.86      0.84      0.85       592
    positive       0.87      0.82      0.84       568

    accuracy                           0.88      1728
   macro avg       0.88      0.88      0.88      1728
weighted avg       0.88      0.88      0.88      1728



In [ ]:
pickle.dump(model, open("sentiment_model.pkl", "wb"))
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))


**Use Case**

In [ ]:

while True:
    user_input = input("\nEnter a financial news headline (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break

    processed_input = clean_text(user_input)
    input_vector = vectorizer.transform([processed_input])
    prediction = model.predict(input_vector)[0]

    print(f"Predicted Sentiment: {prediction}")


Enter a financial news headline (or type 'exit' to quit): Stock market crashes due to inflation!
Predicted Sentiment: neutral

Enter a financial news headline (or type 'exit' to quit): Tech stocks surge after record profits.
Predicted Sentiment: positive

Enter a financial news headline (or type 'exit' to quit): Stock prices plummet as investors fear economic recession
Predicted Sentiment: neutral

Enter a financial news headline (or type 'exit' to quit): exit
